In [1]:
#import packages

import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import string
import torch
import cupy as cp
import numpy as np
import fasttext

In [2]:
# Load the NLTK stop words

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tnaduvil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#pip install fasttext

In [4]:
# Load the pre-trained FastText model

model_path = 'data_in/wiki.en.bin' # # wiki.simple.bin # wiki.en.bin
model = fasttext.load_model(model_path)

## load corpus

In [5]:
#computer_science
lines=[]
with open("data_in/comCont/computer_science_wiki.txt", "r") as file:
    all_lines = file.readlines()
    for line in all_lines:
        line=line.lower()
        line = line.rstrip('\n')  # Remove trailing new line character
        lines.append(line)
print('number of lines is ', len(lines))
lines[0]

number of lines is  69053


'cloud computing[1] is the on-demand availability of computer system resources, especially data storage (cloud storage) and computing power, without direct active management by the user.[2] large clouds often have functions distributed over multiple locations, each of which is a data center.'

In [6]:
#context_lines = context_lines+lines
lines[10]

'as a result, compaq decided to sell server hardware to internet service providers.[7].'

## load term pairs with labels

In [9]:

# Load the .txt file into a Pandas DataFrame
#file_path = 'data_in/glen-master/lenci_zero_glenConstraints.txt'  # Replace with the actual path to your .txt file
#load all 7 datasets vocab to a list.before data.
music= 'data_in/glen-master/music_zero_glenConstraints.txt'
med = 'data_in/glen-master/med_zero_glenConstraints.txt'
cs = 'data_in/glen-master/cs_zero_glenConstraints.txt'
weeds='data_in/glen-master/weeds_zero_glenConstraints.txt'
evaldata='data_in/glen-master/evaldata_zero_glenConstraints.txt'
lenci='data_in/glen-master/lenci_zero_glenConstraints.txt'

#cs = 'data_in/threshold/test/weeds_hypernymy_df_contraintsabsent.csv'
#hypernymy_df = pd.read_csv(cs)
hypernymy_df = pd.read_csv(cs, sep='\t', names=['term1', 'term2', 'hypernymy_relation'])

# Display the DataFrame
hypernymy_df


,term1,term2,hypernymy_relation
0,artificial intelligence,computer science,0
1,computer science,machine learning,1
2,natural language processing,computer science,0
3,computer vision,computer science,0
4,computer science,network security,1
...,...,...,...
242,distributed systems,leader election,1
243,distributed systems,data partitioning,1
244,data replication,distributed systems,0
245,publish-subscribe system,distributed systems,0


In [10]:
hypernymy_relation=hypernymy_df['hypernymy_relation'].to_list()
term1_list = hypernymy_df['term1'].to_list()
term2_list = hypernymy_df['term2'].to_list()

In [11]:

def getSentence(terms):
    zero_sent = []
    term_lines = []
    i = 0
    print(terms)
    for term in terms:
        
        matched_lines = []
        for line in lines:
            
            # Pattern to search for
            pattern = re.escape(term)
            if re.search(pattern, line):
                # Use re.sub() to replace all occurrences of the pattern with an empty string
                modified_text = re.sub(pattern, "", line)
                matched_lines.append(modified_text)

        term_lines.append(matched_lines)
        #print('term number', len(term_lines))
        #print('number of sent found', len(term_lines[i]))
        if len(term_lines[i]) == 0:
            zero_sent.append(term)
            print(zero_sent)
#         else:
#             term_lines[i] = [removeTrueExpressions(line, term) for line in term_lines[i]]
        i = i + 1
    #print(zero_sent)
    return term_lines

In [12]:
#tokenise the sentence with space.

#term1_sentences = [['I love apples', 'apple is healthy'], ['Bananas are yellow'], ['Oranges are citrus fruits']]
def sentTokens(term_sentences):
    tokenized_sentences = []
    for sentence_list in term_sentences:
        sentence_tokens = []
        for sentence in sentence_list:
            tokens = sentence.split()
            sentence_tokens.extend(tokens)
        tokenized_sentences.append(sentence_tokens)

    #print('number of terms : ', len(tokenized_sentences))
    return tokenized_sentences

In [13]:
def stopWordRemove(words_list):
    # Remove stop words from the list of words
    filtered_words = [word for word in words_list if word not in stop_words]
    #remove punctuation
    filtered_words = [word.translate(str.maketrans('', '', string.punctuation)) for word in filtered_words]
    filtered_words = list(filter(None, filtered_words))
    #get unique words
    filtered_words = set(filtered_words)
    filtered_words = list(filtered_words)
    #remove only numeric tokens
    filtered_words = [item for item in filtered_words if not item.isnumeric()]
    #remove if item is single char. it could be initial of authors
    filtered_words = [item for item in filtered_words if len(item) > 1]
    #remove if item doesnt have atleast one alphabet
    filtered_words = [item for item in filtered_words if re.search('[a-zA-Z]', item)]



    
    # Print the filtered words
    #print(filtered_words)
    return filtered_words

In [14]:
## remove meaningless words and get only uniques for term1 and term2. 

def cleanTokens(tokenized_sentences):
    #tokenized_sentences = term1_tokenized_sentences

    cleaned_Tokens = []
    for terms_context_tokens in tokenized_sentences:
        tokens_cleaned = stopWordRemove(terms_context_tokens)
        cleaned_Tokens.append(tokens_cleaned)
    #print(len(cleaned_Tokens))
    return cleaned_Tokens

In [15]:
#get embedings

# Set the device to GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#glove_model.wv.to(device)

def getBertEmbedings(words_list):
    embeding_list = []
    
    for word in words_list:
        # Define input text
        embedding = model.get_word_vector(word)


        # Move embedding to the CPU
        embeding_list.append(embedding.tolist())
      
    
    return list(embeding_list)

In [16]:
#get distance sum of highest c context words
def getDistanceSum(term_uncommon_context):
    #term_uncommon_context = term1_uncommon_context
    distance_sum_list=[]
    i=0
    for context_words in term_uncommon_context:

        if len(context_words) == 0:
            distance_sum_list.append(0)
        else:

            #threshold for number of words
            thresh = min_context_count[i] + 50000
            #thresh = min_context_count[i] + 10000

            if len(context_words) > thresh:
                context_words = context_words[:thresh]
            
            
            #get embedings
            embeding_list = getBertEmbedings(context_words)
            embeding_list = np.array(embeding_list)
            #print('got embed')
            #find distance between each uncommon context embeding and the mean of common context.
            common_context_mean1 = common_context_mean[i]

            # Transfer the arrays to the GPU memory
            gpu_embedding_list = cp.asarray(embeding_list)
            gpu_common_context_mean1 = cp.asarray(common_context_mean1)

            # Compute the Euclidean distances on the GPU
            distances = cp.linalg.norm(gpu_embedding_list - gpu_common_context_mean1, axis=1)

            # Transfer the distances back to the CPU memory as a NumPy array
            distances = cp.asnumpy(distances)

            # Print the resulting distances
            #print('uncommon count',len(distances))
            
            
            #sort the distances in ascending oder
            distances = np.sort(distances)[::-1]
            distance_sum = sum(distances[:min_context_count[i]])
            
            #if min was 0 for its other pair
            if min_context_count[i]==0:
                distance_sum = 10

                      
            distance_sum_list.append(distance_sum)
            #print('sum obtained: ', len(distance_sum_list))
            #print(distance_sum_list)
            

        
        i = i+1

    return distance_sum_list



## DECIDE

In [17]:
predicted_list=[]
hypernymy_relation_list=[]
common_context_count_list=[]
term1_uncommon_context_count_list=[]
term2_uncommon_context_count_list=[]
distance_sum_term1_list=[]
distance_sum_term2_list=[]
term1_cleaned_Tokens_count_list=[]
term2_cleaned_Tokens_count_list=[]
len_term1_sentences_list=[]
len_term2_sentences_list=[]


ind=0

centroid_dist_list = []

for term1_val in term1_list:
    term1=[]
    term2=[]
    #get term2
    term2_val = term2_list[ind]
    term1.append(term1_val)
    term2.append(term2_val)
    

    
    #get sentences
    term1_sentences = getSentence(term1)
    #print(len(term1_sentences))
    term2_sentences = getSentence(term2)
    #print(len(term2_sentences))
    
    #save number of occurence of each term
    len_term1_sentences_list.append(len(term1_sentences[0]))
    len_term2_sentences_list.append(len(term2_sentences[0]))
   
    #tokenise 
    term1_tokenized_sentences = sentTokens(term1_sentences) 
    term2_tokenized_sentences = sentTokens(term2_sentences)
    
    #keep only meaningfull words
    term1_cleaned_Tokens = cleanTokens(term1_tokenized_sentences)
    term2_cleaned_Tokens = cleanTokens(term2_tokenized_sentences)
    
    #get common context
    common_context = [[word for word in sentence1 if word in sentence2] for sentence1, sentence2 in zip(term1_cleaned_Tokens, term2_cleaned_Tokens)]

    #get common_context_count and term1_cleaned_Tokens_count and term2_cleaned_Tokens_count.
    term1_cleaned_Tokens_count = [len(sentence) for sentence in term1_cleaned_Tokens]
    term2_cleaned_Tokens_count = [len(sentence) for sentence in term2_cleaned_Tokens]
    common_context_count = [len(sentence) for sentence in common_context]
    #print('com cont count', common_context_count)

    #uncommon_context_count

    term1_uncommon_context = [[word for word in sentence1 if word not in sentence2] for sentence1, sentence2 in zip(term1_cleaned_Tokens, term2_cleaned_Tokens)]
    term2_uncommon_context = [[word for word in sentence2 if word not in sentence1] for sentence1, sentence2 in zip(term1_cleaned_Tokens, term2_cleaned_Tokens)]
    term1_uncommon_context_count = [len(sentence) for sentence in term1_uncommon_context]
    term2_uncommon_context_count = [len(sentence) for sentence in term2_uncommon_context]
    
    

    common_context_mean=[]
    i=0
    for tok_list in common_context:
        #print(i)
        i=i+1
        embeding_list = getBertEmbedings(tok_list)

        #embeding_list = np.array(embeding_list)
        #mean_vector = np.mean(embeding_list, axis=0)
        embeding_list = cp.asarray(embeding_list)  # Transfer embeding_list to GPU
        mean_vector = cp.mean(embeding_list, axis=0)
        mean_vector = cp.asnumpy(mean_vector)  #

        common_context_mean.append(mean_vector)
    len(common_context_mean)
    
    #c: min (common_context_count, term1_uncommon_context_count)
    min_context_count = [min(a, b) for a, b in zip(term1_uncommon_context_count, term2_uncommon_context_count)]
    
    #calculate dist sum

    distance_sum_term1 =  getDistanceSum(term1_uncommon_context)
    distance_sum_term2 =  getDistanceSum(term2_uncommon_context)

    #predict
    predicted = [1 if a > b else 0 for a, b in zip(distance_sum_term1, distance_sum_term2)]

    #save all
    predicted_list.append(predicted[0])
    
    hypernymy_relation_list.append(hypernymy_relation[ind])
    correct_predictions = sum(a == b for a, b in zip(predicted_list, hypernymy_relation_list))
    accuracy = correct_predictions / len(predicted_list)
    
    
    
    print("Accuracy:", accuracy)
    common_context_count_list.append(common_context_count[0])
    term1_uncommon_context_count_list.append(term1_uncommon_context_count[0])
    term2_uncommon_context_count_list.append(term2_uncommon_context_count[0])
    distance_sum_term1_list.append(distance_sum_term1[0])
    distance_sum_term2_list.append(distance_sum_term2[0])
    term1_cleaned_Tokens_count_list.append(term1_cleaned_Tokens_count[0])
    term2_cleaned_Tokens_count_list.append(term2_cleaned_Tokens_count[0])
    ind=ind+1
    print('#of instances:', ind)
    
    #print('common_context', len(common_context))
    #print('term1_uncommon_context', len(term1_uncommon_context))
    #print('term2_uncommon_context', len(term2_uncommon_context))



##################



['artificial intelligence']
['computer science']
Accuracy: 0.0
#of instances: 1
['computer science']
['machine learning']
Accuracy: 0.0
#of instances: 2
['natural language processing']
['computer science']
Accuracy: 0.3333333333333333
#of instances: 3
['computer vision']
['computer science']
Accuracy: 0.5
#of instances: 4
['computer science']
['network security']
Accuracy: 0.6
#of instances: 5
['data structures']
['computer science']
Accuracy: 0.6666666666666666
#of instances: 6
['computer science']
['operating systems']
Accuracy: 0.5714285714285714
#of instances: 7
['software engineering']
['computer science']
Accuracy: 0.625
#of instances: 8
['computer science']
['database management']
Accuracy: 0.6666666666666666
#of instances: 9
['computer graphics']
['computer science']
Accuracy: 0.7
#of instances: 10
['computer science']
['computer architecture']
Accuracy: 0.7272727272727273
#of instances: 11
['computer science']
['computer networks']
Accuracy: 0.75
#of instances: 12
['computer s

Accuracy: 0.9333333333333333
#of instances: 90
['endpoint protection']
['network security']
Accuracy: 0.9340659340659341
#of instances: 91
['network security']
['application security']
Accuracy: 0.9347826086956522
#of instances: 92
['network security']
['cloud security']
Accuracy: 0.9354838709677419
#of instances: 93
['identity and access management']
['network security']
Accuracy: 0.9361702127659575
#of instances: 94
['network security']
['network address translation']
Accuracy: 0.9368421052631579
#of instances: 95
['physical security']
['network security']
Accuracy: 0.9375
#of instances: 96
['data structures']
['linked lists']
Accuracy: 0.9381443298969072
#of instances: 97
['data structures']
['priority queues']
Accuracy: 0.9387755102040817
#of instances: 98
['hash tables']
['data structures']
Accuracy: 0.9393939393939394
#of instances: 99
['binary search trees']
['data structures']
Accuracy: 0.94
#of instances: 100
['data structures']
['avl trees']
Accuracy: 0.9405940594059405
#of i

Accuracy: 0.9385474860335196
#of instances: 179
['network security']
['computer networks']
Accuracy: 0.9333333333333333
#of instances: 180
['computer networks']
['network topology']
Accuracy: 0.9337016574585635
#of instances: 181
['network protocols']
['computer networks']
Accuracy: 0.9340659340659341
#of instances: 182
['computer networks']
['network devices']
Accuracy: 0.9344262295081968
#of instances: 183
['computer networks']
['network cabling']
Accuracy: 0.9347826086956522
#of instances: 184
['computer networks']
['network architecture']
Accuracy: 0.9351351351351351
#of instances: 185
['network routing']
['computer networks']
Accuracy: 0.9354838709677419
#of instances: 186
['network performance']
['computer networks']
Accuracy: 0.9358288770053476
#of instances: 187
['computer networks']
['network monitoring']
Accuracy: 0.9361702127659575
#of instances: 188
['computer networks']
['network access control']
Accuracy: 0.9365079365079365
#of instances: 189
['computer networks']
['netwo

In [18]:
hypernymy_df['len_term1_sentences'] = len_term1_sentences_list
hypernymy_df['len_term2_sentences'] = len_term2_sentences_list

hypernymy_df['predicted'] = predicted_list

hypernymy_df['term1_uncommon_context_count'] = term1_uncommon_context_count_list
hypernymy_df['term2_uncommon_context_count'] = term2_uncommon_context_count_list

hypernymy_df['term1_cleaned_Tokens_count'] = term1_cleaned_Tokens_count_list
hypernymy_df['term2_cleaned_Tokens_count'] = term2_cleaned_Tokens_count_list

hypernymy_df['distance_sum_term1'] = distance_sum_term1_list
hypernymy_df['distance_sum_term2'] = distance_sum_term2_list

hypernymy_df['common_context_count_list'] = common_context_count_list

hypernymy_df

,term1,term2,hypernymy_relation,len_term1_sentences,len_term2_sentences,predicted,term1_uncommon_context_count,term2_uncommon_context_count,term1_cleaned_Tokens_count,term2_cleaned_Tokens_count,distance_sum_term1,distance_sum_term2,common_context_count_list
0,artificial intelligence,computer science,0,145,152,1,1051,844,1647,1440,3243.029363,3143.482540,596
1,computer science,machine learning,1,152,304,0,826,1628,1440,2242,3104.773507,3440.049570,614
2,natural language processing,computer science,0,47,152,0,283,1200,523,1440,1020.734182,1318.709342,240
3,computer vision,computer science,0,246,152,0,635,1101,974,1440,2311.371025,2633.680236,339
4,computer science,network security,1,152,48,1,1290,267,1440,417,1271.519354,1007.176118,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,distributed systems,leader election,1,24,19,1,208,110,242,144,437.243520,398.486721,34
243,distributed systems,data partitioning,1,24,10,1,235,18,242,25,88.535498,66.703801,7
244,data replication,distributed systems,0,7,24,0,58,227,73,242,200.104523,252.401353,15
245,publish-subscribe system,distributed systems,0,2,24,0,27,235,34,242,109.150036,130.166533,7


In [19]:
#drop the rows which have zero sent in any of term1 and term2 sent
hypernymy_df1 = hypernymy_df[(hypernymy_df['len_term1_sentences'] != 0) & (hypernymy_df['len_term2_sentences'] != 0)]
hypernymy_df1

,term1,term2,hypernymy_relation,len_term1_sentences,len_term2_sentences,predicted,term1_uncommon_context_count,term2_uncommon_context_count,term1_cleaned_Tokens_count,term2_cleaned_Tokens_count,distance_sum_term1,distance_sum_term2,common_context_count_list
0,artificial intelligence,computer science,0,145,152,1,1051,844,1647,1440,3243.029363,3143.482540,596
1,computer science,machine learning,1,152,304,0,826,1628,1440,2242,3104.773507,3440.049570,614
2,natural language processing,computer science,0,47,152,0,283,1200,523,1440,1020.734182,1318.709342,240
3,computer vision,computer science,0,246,152,0,635,1101,974,1440,2311.371025,2633.680236,339
4,computer science,network security,1,152,48,1,1290,267,1440,417,1271.519354,1007.176118,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,distributed systems,leader election,1,24,19,1,208,110,242,144,437.243520,398.486721,34
243,distributed systems,data partitioning,1,24,10,1,235,18,242,25,88.535498,66.703801,7
244,data replication,distributed systems,0,7,24,0,58,227,73,242,200.104523,252.401353,15
245,publish-subscribe system,distributed systems,0,2,24,0,27,235,34,242,109.150036,130.166533,7


In [20]:
hypernymy_df1 = hypernymy_df1[hypernymy_df1['common_context_count_list'] != 0]
hypernymy_df1

,term1,term2,hypernymy_relation,len_term1_sentences,len_term2_sentences,predicted,term1_uncommon_context_count,term2_uncommon_context_count,term1_cleaned_Tokens_count,term2_cleaned_Tokens_count,distance_sum_term1,distance_sum_term2,common_context_count_list
0,artificial intelligence,computer science,0,145,152,1,1051,844,1647,1440,3243.029363,3143.482540,596
1,computer science,machine learning,1,152,304,0,826,1628,1440,2242,3104.773507,3440.049570,614
2,natural language processing,computer science,0,47,152,0,283,1200,523,1440,1020.734182,1318.709342,240
3,computer vision,computer science,0,246,152,0,635,1101,974,1440,2311.371025,2633.680236,339
4,computer science,network security,1,152,48,1,1290,267,1440,417,1271.519354,1007.176118,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,distributed systems,leader election,1,24,19,1,208,110,242,144,437.243520,398.486721,34
243,distributed systems,data partitioning,1,24,10,1,235,18,242,25,88.535498,66.703801,7
244,data replication,distributed systems,0,7,24,0,58,227,73,242,200.104523,252.401353,15
245,publish-subscribe system,distributed systems,0,2,24,0,27,235,34,242,109.150036,130.166533,7


In [21]:
hypernymy_df1.to_csv('data_out/comcont/cs.csv')

In [22]:
hypernymy_df1_correct = hypernymy_df1[hypernymy_df1['hypernymy_relation'] == hypernymy_df1['predicted']]

In [23]:
hypernymy_df1_wrong = hypernymy_df1[hypernymy_df1['hypernymy_relation'] != hypernymy_df1['predicted']]

In [24]:
hypernymy_df1_correct.to_csv('data_out/comcont/cs_corect.csv')
hypernymy_df1_wrong.to_csv('data_out/comcont/cs_wrong.csv')

In [25]:
correct_predictions = (hypernymy_df1['hypernymy_relation'] == hypernymy_df1['predicted']).sum()
total_predictions = len(hypernymy_df1)

accuracy = correct_predictions / total_predictions

print("Accuracy:", accuracy)

Accuracy: 0.8987854251012146
